In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter
pd.set_option("display.precision", 8)

In [2]:
trades = pd.read_csv('../temporary/trades_cleaned.csv', parse_dates = ['date'], infer_datetime_format = True)
trades

,date,tid,price,amount,Currency,amount_nominal,Close,amount_usd,counter
0,2017-03-15 00:00:13,6277278,2.64900700e+04,0.07550000,MXN,2000.00028500,1249.609985,94.34555387,1
1,2017-03-15 00:00:15,100281,4.12698413e+06,0.03150000,VEF,130000.00009500,1249.609985,39.36271453,1
2,2017-03-15 00:00:15,6277265,1.29297000e+03,0.65740000,USD,849.99847800,1249.609985,821.49360414,1
3,2017-03-15 00:00:24,6277292,1.06800000e+03,0.11000000,GBP,117.48000000,1249.609985,137.45709835,1
4,2017-03-15 00:00:40,6277296,1.06293000e+03,0.11760000,GBP,125.00056800,1249.609985,146.95413424,1
...,...,...,...,...,...,...,...,...,...
42636057,2022-09-28 01:55:04,56510551,8.40412643e+07,0.00285574,COP,239999.99999785,19426.720703,55.47766338,1
42636058,2022-09-28 01:55:10,56510558,1.80295800e+04,0.03189204,USD,575.00008654,19426.720703,619.55775373,1
42636059,2022-09-28 01:55:10,56510556,2.16994700e+04,0.00553009,USD,120.00002205,19426.720703,107.43151389,1
42636060,2022-09-28 01:57:02,56510553,2.00452200e+04,0.00249436,USD,49.99999496,19426.720703,48.45723505,1


## Get Distribution of Trade Sizes

In [3]:
freq = Counter(trades['amount'])
trade_sizes = pd.DataFrame(freq.items(), columns = ['trade_size', 'frequency'])
trade_sizes.sort_values(by = 'frequency')
trade_sizes

,trade_size,frequency
0,0.07550000,106
1,0.03150000,414
2,0.65740000,1
3,0.11000000,2384
4,0.11760000,46
...,...,...
7413009,0.13503517,1
7413010,0.02979257,1
7413011,0.07398786,1
7413012,0.15726642,1


In [4]:
K = len(trades) #gives amount of total trades ~40million
trade_sizes['p'] = trade_sizes['frequency']/K #unsure if this is right way to do it, but sum(p) = 1
trade_sizes

,trade_size,frequency,p
0,0.07550000,106,0.00000249
1,0.03150000,414,0.00000971
2,0.65740000,1,0.00000002
3,0.11000000,2384,0.00005592
4,0.11760000,46,0.00000108
...,...,...,...
7413009,0.13503517,1,0.00000002
7413010,0.02979257,1,0.00000002
7413011,0.07398786,1,0.00000002
7413012,0.15726642,1,0.00000002


In [5]:
trade_sizes[trade_sizes['trade_size'] == 0.00202160]

,trade_size,frequency,p
813522,0.0020216,34,0.0000008


# Match Trades

In [27]:
#'data' must have the columns to be copied, and 'window' must be in the form of hours
def get_trades(data, window):
    
    t = str(window) + 'h'
    
    data.sort_values(['amount', 'date'], inplace=True)
    data['tid2'] = data['tid'].shift(-1)
    data['date2'] = data['date'].shift(-1)
    data['amount2'] = data['amount'].shift(-1)
    data['currency2'] = data['currency'].shift(-1)
    data['amount_nominal2'] = data['amount_nominal'].shift(-1)
    data['time_diff'] = data['date2'] - data['date']
    df = data[(data['time_diff'] <= (t)) & (data['amount'] == data['amount2'])]
    
    return df


#'data' must have the columns to be copied, and 'window' must be in the form of hours
def get_trades_paxful(trades, window):
    
    t = str(window) + 'h'

    data.sort_values(['amount', 'date'], inplace=True)
    data['tid2'] = data['tid'].shift(-1)
    data['date2'] = data['date'].shift(-1)
    data['amount2'] = data['amount'].shift(-1)
    data['currency2'] = data['currency'].shift(-1)
    data['type2'] = data['type'].shift(-1)
    data['payment_method2'] = data['payment_method'].shift(-1)
    data['time_diff'] = data['date2'] - data['date']    
    data['user_cc2'] = data['user_cc'].shift(-1)
    df = data[(data['time_diff'] <= (t)) & (data['amount'] == data['amount2'])]
    
    return df

In [19]:
matched_trades = get_trades(trades, 5)
matched_trades = matched_trades.sort_values('date')

NameError: name 'trades' is not defined

In [5]:
matched_trades

,date,tid,price,amount,Currency,amount_nominal,Close,amount_usd,counter,tid2,date2,amount2,Currency2,amount_nominal2,time_diff
3,2017-03-15 00:00:24,6277292,1068.00,0.11000000,GBP,117.48000000,1249.609985,137.45709835,1,100294.0,2017-03-15 00:16:23,0.11000000,VEF,456500.00000000,0 days 00:15:59
4,2017-03-15 00:00:40,6277296,1062.93,0.11760000,GBP,125.00056800,1249.609985,146.95413424,1,100435.0,2017-03-15 03:40:44,0.11760000,VEF,499999.99996800,0 days 03:40:04
7,2017-03-15 00:00:49,6277295,547460.29,0.15990000,NGN,87538.90037100,1249.609985,199.81263660,1,6277321.0,2017-03-15 00:01:37,0.15990000,USD,225.00008700,0 days 00:00:48
8,2017-03-15 00:00:58,6277291,1526.72,0.03930000,USD,60.00009600,1249.609985,49.10967241,1,6277338.0,2017-03-15 00:06:06,0.03930000,USD,60.00009600,0 days 00:05:08
11,2017-03-15 00:01:14,6277315,78740.16,0.01270000,RUB,1000.00003200,1249.609985,15.87004681,1,6277362.0,2017-03-15 00:12:30,0.01270000,RUB,1000.00003200,0 days 00:11:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42635627,2022-09-27 22:44:19,56510120,84277182.39,0.00108333,COP,91299.99999856,19110.546875,20.70302875,1,56510151.0,2022-09-27 22:52:11,0.00108333,COP,91215.30000501,0 days 00:07:52
42635745,2022-09-27 23:28:04,56510220,158900.00,0.00200000,VED,317.80000000,19110.546875,38.22109375,1,56510533.0,2022-09-28 01:42:17,0.00200000,CRC,24465.45000000,0 days 02:14:13
42635795,2022-09-27 23:46:13,56510295,1227596.37,0.00040730,RUB,500.00000150,19110.546875,7.78372574,1,56510426.0,2022-09-28 00:46:25,0.00040730,RUB,500.00000150,0 days 01:00:12
42635845,2022-09-28 00:10:07,56510348,19417017.91,0.00721017,CLP,140000.00002414,19426.720703,140.06995881,1,56510346.0,2022-09-28 00:13:07,0.00721017,CLP,140000.00002414,0 days 00:03:00


In [6]:
matched_trades.to_csv('../temporary/matched_trades.csv', index=False)

# Paxful

In [32]:
trades_paxful = pd.read_csv('../temporary/trades_paxful_cleaned.csv',
                            dtype={
                                'tid':int,
                                'amount':float,
                                'price':float,
                                'crypto_rate_usd':float,
                                'amount_usd':float
                            },
                            parse_dates = ['date'],
                            infer_datetime_format = True
                           )

In [33]:
trades_paxful.head()

,tid,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,amount_usd
0,4544629,2017-03-15 06:04:04,0.01306959,20.0,"Any Visa, MasterCard and AmEx Gift Cards",Gift cards,USD,buy,US,US,1246.86,16.29594899
1,4544627,2017-03-15 06:04:26,0.05728689,100.0,PayPal,Online wallets,USD,buy,CM,US,1246.86,71.42873167
2,4544639,2017-03-15 06:04:39,0.01688459,25.0,Amazon Gift Card,Gift cards,USD,buy,US,US,1247.49,21.06335718
3,4544613,2017-03-15 06:05:27,0.09403504,150.0,iTunes Gift Card,Gift cards,USD,buy,CN,ZA,1246.86,117.24852997
4,4544617,2017-03-15 06:05:28,0.59586015,1300.0,Amazon Gift Card,Gift cards,USD,sell,US,UZ,1246.86,742.95418663


In [34]:
matched_paxful_trades = get_trades_paxful(trades_paxful, 5)
matched_paxful_trades = matched_paxful_trades.sort_values('date')

In [35]:
matched_paxful_trades.head()

,tid,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,amount_usd,tid2,date2,amount2,currency2,type2,payment_method2,time_diff,user_cc2
7,4544638,2017-03-15 06:06:46,0.03132871,50.0,iTunes Gift Card,Gift cards,USD,buy,KR,NG,1247.49,39.08225244,4544650.0,2017-03-15 06:08:06,0.03132871,USD,buy,iTunes Gift Card,0 days 00:01:20,NG
17,4544664,2017-03-15 06:10:00,0.03136448,50.0,iTunes Gift Card,Gift cards,USD,buy,CN,US,1247.49,39.12687516,4544667.0,2017-03-15 06:10:40,0.03136448,USD,buy,iTunes Gift Card,0 days 00:00:40,US
37,4544739,2017-03-15 06:16:11,0.03135386,50.0,iTunes Gift Card,Gift cards,USD,buy,CN,NG,1247.91,39.12679543,4544727.0,2017-03-15 06:16:56,0.03135386,USD,buy,iTunes Gift Card,0 days 00:00:45,NG
41,4544712,2017-03-15 06:16:27,0.02837799,42.0,Amazon Gift Card,Gift cards,USD,buy,PH,US,1247.91,35.41317750,4544762.0,2017-03-15 06:20:40,0.02837799,USD,buy,Amazon Gift Card,0 days 00:04:13,US
42,4544727,2017-03-15 06:16:56,0.03135386,50.0,iTunes Gift Card,Gift cards,USD,buy,CN,NG,1247.91,39.12679543,4544743.0,2017-03-15 06:20:18,0.03135386,USD,buy,iTunes Gift Card,0 days 00:03:22,NG


In [36]:
path = '../temporary/matched_paxful_trades.csv'
matched_paxful_trades.to_csv(path, index = False)

In [26]:
matched_paxful_trades[matched_paxful_trades['currency'] != matched_paxful_trades['currency2']]

,tid,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,...,amount_usd,tid2,date2,amount2,currency2,time_diff,type2,payment_method2,user_cc2,advertiser_cc2
451,4545569,2017-03-15 07:50:58,0.03135484,50.00,iTunes Gift Card,Gift cards,USD,buy,US,US,...,39.22741323,4546157.0,2017-03-15 09:21:16,0.03135484,NGN,0 days 01:30:18,sell,Bank Transfer,NG,NG
661,4545969,2017-03-15 08:48:28,0.01566681,25.00,iTunes Gift Card,Gift cards,USD,buy,IN,US,...,19.61923283,4546167.0,2017-03-15 09:09:05,0.01566681,NGN,0 days 00:20:37,sell,Bank Transfer,US,NG
727,4546199,2017-03-15 09:10:09,0.05863582,95.00,iTunes Gift Card,Gift cards,USD,buy,US,NG,...,73.35927440,4546323.0,2017-03-15 10:27:22,0.05863582,NGN,0 days 01:17:13,sell,Bank Transfer,NG,KE
1031,4546861,2017-03-15 10:58:07,0.00230000,3.51,PayPal,Online wallets,AUD,sell,LK,AU,...,2.86918100,4547657.0,2017-03-15 12:39:40,0.00230000,NZD,0 days 01:41:33,sell,PayPal,AU,LK
1164,4547412,2017-03-15 11:53:39,0.03205478,50.00,iTunes Gift Card,Gift cards,USD,buy,US,GB,...,40.07071883,4547583.0,2017-03-15 13:28:41,0.03205478,NGN,0 days 01:35:02,sell,Bank Transfer,NG,NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89665709,200219803,2022-09-29 04:37:07,0.00146707,40.00,eBay Gift Card,Gift cards,USD,buy,VN,NG,...,28.67290021,200220087.0,2022-09-29 04:50:31,0.00146707,NGN,0 days 00:13:24,sell,Bank Transfer,NG,NG
89665729,200210153,2022-09-29 04:38:13,0.00162496,23184.00,Bank Transfer,Bank transfers,NGN,sell,NG,NG,...,31.69983343,200104810.0,2022-09-29 05:09:47,0.00162496,ZAR,0 days 00:31:34,sell,FNB E-WALLET,ZA,ZA
89665985,200220412,2022-09-29 04:50:16,0.00054547,7673.20,Bank Transfer,Bank transfers,NGN,sell,NG,NG,...,10.67491881,200221261.0,2022-09-29 05:07:54,0.00054547,KES,0 days 00:17:38,buy,M-Pesa,KE,KE
89666583,200221690,2022-09-29 05:17:13,0.00500000,70000.00,Bank Transfer,Bank transfers,NGN,sell,NG,GB,...,97.86030000,200211288.0,2022-09-29 07:00:28,0.00500000,USD,0 days 01:43:15,sell,NIP (NIBSS Instant Payment),NG,NG


# Code Testing